**LABORATORIO 6**
* repositorio github:https://github.com/R1mber/sis421/tree/master/Laboratorios
---

* Dataset:https://drive.google.com/file/d/12J1Nav_zWQyBMhxu48jyZ-FA_zSQxLk3/view?usp=sharing

In [17]:
import os
import torch
import random
import torchvision
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from PIL import Image
from skimage import io


import pandas_datareader as web
import pandas as pd
import datetime as dt

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:
cp "/content/drive/MyDrive/inteligencia_artificial/ia2/laboratorio/laboratorio 6/NFLX.csv" "/content/NFLX.csv"

In [29]:
data=pd.read_csv('NFLX.csv')
data

,Date,Open,High,Low,Close,Adj Close,Volume
0,2015-12-16,119.800003,123.000000,118.089996,122.639999,122.639999,13181000
1,2015-12-17,123.970001,126.349998,122.419998,122.510002,122.510002,17284900
2,2015-12-18,120.849998,122.190002,117.919998,118.019997,118.019997,17948100
3,2015-12-21,119.510002,119.589996,115.660004,116.629997,116.629997,11670000
4,2015-12-22,117.300003,117.430000,114.860001,116.239998,116.239998,9689000
...,...,...,...,...,...,...,...
1002,2019-12-10,296.119995,298.940002,292.019989,293.119995,293.119995,10476100
1003,2019-12-11,294.489990,299.429993,294.200012,298.929993,298.929993,5589800
1004,2019-12-12,295.670013,299.170013,295.059998,298.440002,298.440002,4766600
1005,2019-12-13,298.500000,301.799988,297.250000,298.500000,298.500000,3879700


In [36]:
n=len(data)
train_data=data[0:(n//10)*8]
test_data=data[(n//10)*8:(n//10)*9]
eval_data=data[(n//10)*9:]

In [37]:
len(data), len(train_data), len(test_data), len(eval_data)

(1007, 800, 100, 107)

In [33]:
#normalizar datos
'''
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(train_data['Open'].values.reshape(-1,1))
scaled_data.shape 
'''

(800, 1)

In [67]:
scaled_data_train = (train_data['Open'].values.reshape(-1,1))
scaled_data_test = (test_data['Open'].values.reshape(-1,1))
scaled_data_eval = (eval_data['Open'].values.reshape(-1,1))
scaled_data_train.shape, scaled_data_test.shape, scaled_data_eval.shape

((800, 1), (100, 1), (107, 1))

In [81]:
def getXandY(dias,scaled_data):
    x = []
    y = []
    for xi in range(dias, len(scaled_data)-10):
        x.append(scaled_data[xi-dias:xi, 0])
        y.append(scaled_data[xi+10, 0]) #proxiomos 10 para prediccion

    x, y = np.array(x), np.array(y)
    x = np.reshape(x, (x.shape[0], x.shape[1], 1))
    y = np.reshape(y, (y.shape[0], 1))
    return x, y

In [82]:
dias = 30
x_train, y_train = getXandY(dias, scaled_data_train)
x_test, y_test = getXandY(dias, scaled_data_test)
x_eval, y_eval = getXandY(dias, scaled_data_eval)

In [83]:
x_train.shape,y_train.shape , x_test.shape

((760, 30, 1), (760, 1), (60, 30, 1))

In [84]:
import torch
from torch.utils.data import Dataset, DataLoader

class TimeSeriesDataset(Dataset):
  def __init__(self, X, y=None, train=True):
    self.X = X
    self.y = y
    self.train = train

  def __len__(self):
    return len(self.X)

  def __getitem__(self, ix):
    if self.train:
      return torch.from_numpy(self.X[ix]), torch.from_numpy(self.y[ix])
    return torch.from_numpy(self.X[ix])

dataset = {
    'train': TimeSeriesDataset(x_train, y_train),
    'eval': TimeSeriesDataset(x_eval, y_eval),
    'test': TimeSeriesDataset(x_test, y_test, train=False)
}

dataloader = {
    'train': DataLoader(dataset['train'], shuffle=True, batch_size=64),
    'eval': DataLoader(dataset['eval'], shuffle=False, batch_size=64),
    'test': DataLoader(dataset['test'], shuffle=False, batch_size=64)
}

In [85]:
class RNN(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.rnn = torch.nn.RNN(input_size=1, hidden_size=20, num_layers=1, batch_first=True)
    self.fc = torch.nn.Linear(20, 1)

  def forward(self, x):
    x, h = self.rnn(x) 
    y = self.fc(x[:,-1])
    return y

rnn = RNN()

In [87]:
from tqdm import tqdm
device = "cuda" if torch.cuda.is_available() else "cpu"

def fit(model, dataloader, epochs=10):
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = torch.nn.MSELoss()
    bar = tqdm(range(1, epochs+1))
    for epoch in bar:
        model.train()
        train_loss = []
        for batch in dataloader['train']:
            X, y = batch
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            y_hat = model(X)
            loss = criterion(y_hat, y)
            loss.backward()
            optimizer.step()
            train_loss.append(loss.item())
        model.eval()
        eval_loss = []
        with torch.no_grad():
            for batch in dataloader['eval']:
                X, y = batch
                X, y = X.to(device), y.to(device)
                y_hat = model(X)
                loss = criterion(y_hat, y)
                eval_loss.append(loss.item())
        bar.set_description(f"loss {np.mean(train_loss):.5f} val_loss {np.mean(eval_loss):.5f}")
        
def predict(model, dataloader):
    model.eval()
    with torch.no_grad():
        preds = torch.tensor([]).to(device)
        for batch in dataloader:
            X = batch
            X = X.to(device)
            pred = model(X)
            preds = torch.cat([preds, pred])
        return preds

In [88]:
fit(rnn, dataloader)

  0%|          | 0/10 [00:00<?, ?it/s]


RuntimeError: ignored

In [90]:
class DeepRNN(torch.nn.Module):
  def __init__(self, n_in=50, n_out=10):
    super().__init__()
    self.rnn = torch.nn.RNN(input_size=1, hidden_size=20, num_layers=2, batch_first=True)
    self.fc = torch.nn.Linear(20, 1)

  def forward(self, x):
    x, h = self.rnn(x) 
    x = self.fc(x[:,-1])
    return x

In [91]:
rnn = DeepRNN()
fit(rnn, dataloader)

  0%|          | 0/10 [00:00<?, ?it/s]


RuntimeError: ignored